
#The following instructions can be used to train a Transformer model on the [IWSLT'14 German to English dataset](http://workshop2014.iwslt.org/downloads/proceeding.pdf).


In [ ]:

%%bash 

cd /mnt/c/translator
cd examples/translation/
bash prepare-wmt14en2de.sh
cd ../..

TEXT=examples/translation/wmt17_en_de
fairseq-preprocess \
    --source-lang en --target-lang de \
    --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test \
    --destdir data-bin/wmt17_en_de --thresholdtgt 0 --thresholdsrc 0 \
    --workers 20
    
mkdir -p checkpoints/fconv_wmt_en_de
# fairseq-train \
#     data-bin/wmt17_en_de \
#     --arch fconv_wmt_en_de \
#     --dropout 0.2 \
#     --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
#     --optimizer nag --clip-norm 0.1 \
#     --lr 0.5 --lr-scheduler fixed --force-anneal 50 \
#     --max-tokens 4000 \
#     --save-dir checkpoints/fconv_wmt_en_de
# alias python=python3


    
# cd examples/translation/
# bash wmt14en-de.sh
# cd ../..

# # Preprocess/binarize the data
# TEXT=examples/translation/wmt14en2de.tokenized.de-en
# fairseq-preprocess --source-lang de --target-lang en \
#     --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test \
#     --destdir data-bin/wmt14en2de.tokenized.de-en \
#     --workers 20

: 

In [ ]:
%%bash 
CUDA_VISIBLE_DEVICES=0 fairseq-train \
    data-bin/wmt17_en_de \
    --arch meta-transformer_wmt_de_en --share-decoder-input-output-embed \
    --user-dir models/transformer_legacy.py\
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric

In [ ]:
%%bash
fairseq-generate data-bin/iwslt14.tokenized.de-en \
    --path checkpoints/checkpoint_best.pt \
    --batch-size 128 --beam 5 --remove-bpe